## Packages Installation

In [ ]:
!pip install gym_super_mario_bros

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.0 MB/s eta 0:00:00
  Created wheel for nes-py: filename=nes_py-8.2.1-cp39-cp39-linux_x86_64.whl size=494939 sha256=b82900586a8df651dbcc2065c378685d6f8e33a494a7aa87bf66dc393ec41c52
  Stored in directory: /root/.cache/pip/wheels/c6/e1/4b/dbbd5d4a46ad80c0149d5671edb272c728c130e4d5750ca1d2
Successfully built nes-py


## Importing Required Libraries

In [ ]:
import cv2, gym, torch, math, random
import torch.nn as nn
from collections import deque
import numpy as np
import torch.optim as optim
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

## Making a Custom Model Class

In [ ]:
# define the model
class LearningAgent(nn.Module):
    def __init__(self):
        super(LearningAgent, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.rnn = nn.LSTM(7*7*64, 512, batch_first=True)
        self.fc = nn.Linear(512, 5)
        
    # forward pass
    def forward(self, x, hx=None):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.relu(self.conv3(x))
        x = x.view(-1, 7*7*64)
        if hx is not None:
            hx = (hx[0].contiguous(), hx[1].contiguous())
            x, hx = self.rnn(x.unsqueeze(0), hx)
            x = self.fc(x.squeeze(0))
            return x, hx
        else:
            x, _ = self.rnn(x.unsqueeze(0))
            x = self.fc(x.squeeze(0))
            return x

## Defining Model Parameters
The parameters of reinforcement learning model should be defined before calling a model.

In [ ]:
# defining the training parameters grid
m_params = {
    'lr': 0.005,
    'discount_factor': 0.94,
    'number_of_episodes': 200,
    'batch_size': 128,
    'update_frequency': 5,
    'target_update_frequency': 200
}

# assigning values to variables using parameter grid
lr = m_params['lr']
gamma = m_params['discount_factor']
num_episodes = m_params['number_of_episodes']
batch_size = m_params['batch_size']
update_frequency = m_params['update_frequency']
target_update_frequency = m_params['target_update_frequency']

## Setting Up the Game Environment

In [ ]:
# creating the environment
env = gym.make('SuperMarioBros-v0')

# creating a learning agent model and optimization function
model = LearningAgent()

# setting the optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

/usr/local/lib/python3.9/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


## Defining a Loss Function

In [ ]:
# defining a loss function
def loss_function_calculate(state, action, reward, 
                            next_state, done):
    with torch.no_grad():
        # calculating the Q-values for the current state and action
        q_values, _ = model(state)
        q_value = q_values.gather(1, action.unsqueeze(1))

        # calculating the Q-values for the next state
        next_q_values, _ = target_model(next_state)
        next_q_value, _ = torch.max(next_q_values, 
                                    1, keepdim=True)

        # calculating the target Q-value
        target_q_value = reward + gamma * next_q_value * (1 - done)

    # calculating the mean-squared-error (MSE) loss
    final_loss = nn.functional.mse_loss(q_value, target_q_value)

    return final_loss

## Using Epilon-Greedy Policy

In [ ]:
# defining the epsilon-greedy exploration policy
epsi_start = 1.0
epsi_end = 0.1
epsi_decay = 10000
epsi_by_frame = lambda frame_idx: epsi_end + (epsi_start - epsi_end) * \
    math.exp(-1. * frame_idx / epsi_decay)

### Coding Replay Buffer Class

In [ ]:
# This custom class will reply the buffer size
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        states, actions, rewards, next_states, dones = zip(*random.sample(self.buffer, batch_size))
        return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)
    
    def __len__(self):
        return len(self.buffer)

## Replaying Buffer Class

In [ ]:
# define the replay buffer
buffer_replaying = ReplayBuffer(1000)

# target network
target_model = LearningAgent()
target_model.load_state_dict(model.state_dict())
target_model.eval()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LearningAgent(
  (conv1): Conv2d(64, 64, kernel_size=(8, 8), stride=(4, 4))
  (conv2): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2))
  (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (rnn): LSTM(3136, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=5, bias=True)
)

In [ ]:
# defining the frame counter
frame_idx = 0

## Applying Training Loop

In [ ]:
for episode in range(num_episodes):
    state = env.reset()
    episode_reward = 0
    done = False
    hx = None

    while not done:
        # choosing an action using the epsilon-greedy policy
        if random.random() < epsilon_by_frame(frame_idx):
            action = env.action_space.sample()
        else:
          state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
          action_tensor = torch.tensor([action], dtype=torch.long)
          next_state, reward, done, _ = env.step(action)
          replay_buffer.push(state, action, reward, next_state, done)
          state = next_state
          episode_reward += reward
          frame_idx += 1

           # sample a batch of transitions from the replay buffer
        if len(replay_buffer) > batch_size:
            state_batch, action_batch, reward_batch, next_state_batch, done_batch = \
                replay_buffer.sample(batch_size)

            # calculate the loss and update the model
            loss = compute_loss(
                torch.tensor(state_batch, dtype=torch.float32),
                torch.tensor(action_batch, dtype=torch.long),
                torch.tensor(reward_batch, dtype=torch.float32),
                torch.tensor(next_state_batch, dtype=torch.float32),
                torch.tensor(done_batch, dtype=torch.float32)
            )
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update the target network
            if frame_idx % target_update_frequency == 0:
                target_model.load_state_dict(model.state_dict())
    # print the episode results
    print(f"Episode {episode}: reward = {episode_reward}")

    # save the model
    if (episode + 1) % 100 == 0:
        torch.save(model.state_dict(), f"model_episode_{episode+1}.pt")